In [1]:
from Levenshtein import distance as levenshtein_distance
from time import *
import time
import numpy as np
import math
import random
import os

#from itertools import combinations, combinations_with_replacement
#from collections import Counter
#from io import StringIO 
#import sys
#import re
#import swalign
#%matplotlib inline
#from matplotlib_venn import venn2, venn3

# Division Kmer

In [2]:
def kmers_split(seq, k=5, taille_V = 280, nb_indice = 70):
    dict1 = []
    # on divise nos k-mers en sous partie. le but est de comparer tous les k-mers avec le meme indice=div_counter
    # dans la sequence query avec tous les k-mers avec le meme indice dans la seq ref
    # cela permettra d eviter a comparer les k-mers en premier avec des k-mers en dernier
    
    #nb_indice: exemple -> div = int(280/70) = 4, alors pr chaque 4 kmers on fait +1 a l indice 
    div = int(taille_V/nb_indice)
    div_fix = int(taille_V/nb_indice)
    div_counter = 0
    for i in range(len(seq)-(k-1)):
        if i == div:
            div_counter += 1
            div += div_fix
        dict1.append([seq[i:i+k], div_counter])   
    return dict1

In [3]:
# test
kmers_split("CGACAAACGTCG", 5, 280)

[['CGACA', 0],
 ['GACAA', 0],
 ['ACAAA', 0],
 ['CAAAC', 0],
 ['AAACG', 1],
 ['AACGT', 1],
 ['ACGTC', 1],
 ['CGTCG', 1]]

In [4]:
# test2
#kmers_split("CGACAAACGTCGCGACAAACGTCG", 5, 280)

#### 1er petit Test sur les données

In [5]:
query_s2 ="CAGGGACAGCTGGTGCAGTCTGGGGCTGAGGAGAAGAGGCCTGGGGCCTCAGTGAAAGTCTCCTGCAAGGCCTCTGGATACACCATCACCAGCTACTGTATGCACTGGGTCCACCAGGTCCATGCACAAGGGCTTGAGTGGATGGGATTGGTGTGCCCTAGTGATGGCAGCACAAGCTATGCACAGAAGTTCCAGGCCAGAGTCACCATAACCAGGGACACATCCATGAGCACAGCCTACATGGAGCTAAGCAGTCTGAGATCTGAGGACACGGCCATGTATTACTGTGTGAGACGCCAGCTATGGGAGACTGATTGCTTCCAGCACTGGGGCCAGGTCACCCTGGTCACTGTCTCCTCAG"
ref_v1_or21 = "caggtacagctggtgcagtctggggctgaggtgaagaagcctggggcctcagtgaaggtctcctgcaaggcttctggatacaccatcaccagctactgtatgcactgggtgcaccaggtccatgcacaagggcttgagtggatgggattggtgtgccctagtgatggcagcacaagctatgcacagaagttccaggccagagtcaccataaccagggacacatccatgagcacagcctacatggagctaagcagtctgagatctgaggacacggccatgtattactgtgtgagaga"
ref_v1_18_01 = "caggttcagctggtgcagtctggagctgaggtgaagaagcctggggcctcagtgaaggtctcctgcaaggcttctggttacacctttaccagctatggtatcagctgggtgcgacaggcccctggacaagggcttgagtggatgggatggatcagcgcttacaatggtaacacaaactatgcacagaagctccagggcagagtcaccatgaccacagacacatccacgagcacagcctacatggagctgaggagcctgagatctgacgacacggccgtgtattactgtgcgagaga"

In [6]:
r_kmer = kmers_split(ref_v1_or21.upper(), 5)
r_kmer2 = kmers_split(ref_v1_18_01.upper(), 5)
q_kmer = kmers_split(query_s2, 5)
#print(r_kmer)

# Comparaison des listes de kmer de la forme [[kmer,indice]]

In [7]:
import time
def comparaison(query_kmer, ref_kmer, bloc_max = 40):
    count = 0 # pour le nombre de [kmer, indice] identique dans les deux sequences
    block = 0 # pour areter l'algorithme si pour un nombre 'block' on n'a pas trouver des [kmer, indice] identique
    query_kmer_copy = query_kmer.copy()
    
    for kmer in ref_kmer: # on parcout les [kmer, indice] de la sequence reference
        if kmer in query_kmer_copy: # on voit si ces [kemr, indice] sont dans la sequence query
            count += 1 # on precise qu'on a trouve un [kmer, indice]
            block = 0 # on remet a 0 le block vu qu'on a trouver un [kemr, indice]
            query_kmer_copy.remove(kmer)
                     
        if block == bloc_max:
            return count
        
        
        block += 1

    return count
debut = time.time()
comparaison(q_kmer, r_kmer)
comparaison(q_kmer, r_kmer)
comparaison(q_kmer, r_kmer)
fin = time.time()
print("temps:", fin-debut)

temps: 0.00449824333190918


#### 2eme petit Test sur les données

In [8]:
query_IGHV3_30_3_01 = "CAAGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAATAGCAATGCTATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCAGTTATATCATATGATGGAAGAAATAGATACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCTGAGGACACGGCTGTGTATTACTGTGCGAGCGTTAAATGTGGATATAGTGGCTACGATTACACATACTTCCAGCACTGGGGCCAGGGCACCCTGGTCACCGTCTCCTCAG"
ref_IGHV3_30_3_01 = "caggtgcagctggtggagtctgggggaggcgtggtccagcctgggaggtccctgagactctcctgtgcagcctctggattcaccttcagtagctatgctatgcactgggtccgccaggctccaggcaaggggctggagtgggtggcagttatatcatatgatggaagcaataaatactacgcagactccgtgaagggccgattcaccatctccagagacaattccaagaacacgctgtatctgcaaatgaacagcctgagagctgaggacacggctgtgtattactgtgcgagaga"
ref_IGHV3_30_04 = "caggtgcagctggtggagtctgggggaggcgtggtccagcctgggaggtccctgagactctcctgtgcagcctctggattcaccttcagtagctatgctatgcactgggtccgccaggctccaggcaaggggctggagtgggtggcagttatatcatatgatggaagtaataaatactacgcagactccgtgaagggccgattcaccatctccagagacaattccaagaacacgctgtatctgcaaatgaacagcctgagagctgaggacacggctgtgtattactgtgcgagaga"

q_kmer2 = kmers_split(query_IGHV3_30_3_01)
r_kmer2 = kmers_split(ref_IGHV3_30_3_01.upper())
r_kmer22 = kmers_split(ref_IGHV3_30_04.upper())

#print(comparaison(q_kmer2, r_kmer2))
#print(comparaison(q_kmer2, r_kmer22))

# Etude sur nos Fichiers

In [9]:
# lecture du fichier reference
def read_db(path):
    dic_db = {}
    f = open(path, 'r')
    while 1:
        line = f.readline().strip()
        if line == '':
            break
        if line[0] == '>':
            fa = f.readline().strip()
            fa = fa.upper()
            dic_db[line[1:]] = fa
    f.close()
    return dic_db

In [10]:
# extraction des genes ref
db_path = "human_germline//IGHV.fasta"
dic_db = dict(read_db(db_path))
print(dic_db['IGHV1-18*01'])

CAGGTTCAGCTGGTGCAGTCTGGAGCTGAGGTGAAGAAGCCTGGGGCCTCAGTGAAGGTCTCCTGCAAGGCTTCTGGTTACACCTTTACCAGCTATGGTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAGCGCTTACAATGGTAACACAAACTATGCACAGAAGCTCCAGGGCAGAGTCACCATGACCACAGACACATCCACGAGCACAGCCTACATGGAGCTGAGGAGCCTGAGATCTGACGACACGGCCGTGTATTACTGTGCGAGAGA


In [11]:
## extraction des genes query
# ouvrir le fichier query et mettre tous les elemens v dans un dictionnaire ayant comme cle nom du v 
# et comme valeur la sequence

def extract_vquery(fichier):
    name_fichier_query = fichier
    f = open(name_fichier_query,'r')
    i = 0
    #dict_v = {}
    dict_vquery_keys = []
    dict_vquery_values = []
    while True:
        # lecture du fichier query
        line = f.readline().strip()
        #print(line)
        if line == '':
            break
        # extraction des sequences ainsi que le nom du bloc D
        li = line
        if li[0] == '>':
            tmp_d = li.split('|')[1]

            #dict_v[tmp_d] = ''
            dict_vquery_keys.append(tmp_d)
            continue
        
        #dict_v[tmp_d] = li.upper()
        dict_vquery_values.append(li.upper())
        i += 1
    f.close()
        
    return dict_vquery_keys, dict_vquery_values
name_fichier_query = 'VDJ-Kernel-main//data//datasets//simulations//simple_plus_indels_40Mut_out.fasta'
#dict_vquery = extract_vquery(name_fichier_query)
#print(len(dict_vquery))
#print(dict_vquery['IGHV3-30-3*01'])
#print(dict_vquery['IGHV5-51*02'])

dict_vquery_keys, dict_vquery_values = extract_vquery(name_fichier_query)

print(len(dict_vquery_keys))
print(len(dict_vquery_values))

10000
10000


In [32]:
# fonction qui nous permet de comparer les [kemrs, indice] des seq ref avec les [kmer, indice] 
# d'une seule seq query
def compare_fichier(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, k=5, taille_V = 280):
    dic = {}
    for j in range(len(dic_db_keys)):
        r_kmer = kmers_split(dic_db_values[j], 5, 280, nb_indice) # on divise en [kmer, indice] la seq ref
        comp = comparaison(r_kmer, q_kmer, bloc_max ) # on realise la comparaison
        cle = dic_db_keys[j]
        dic[cle] = comp
            
    return dic

In [33]:
# dans le cas où notre algo a eu 2 ou plus sequences de meme score, on fait un test de plus, qui est celui de
# levenshtein et on reduit le nom des sequences en enlevant les allèles. On considere ainsi que
# 'IGHV3-30*04', 'IGHV3-30*09' sont identiques
def ref_in_listoOfKeys(listOfKeys, query):
    dict_count = {}
    for v_ref in listOfKeys:
        distance = levenshtein_distance(v_ref, query )
        try:
            pos = v_ref.index('*')
            cle = v_ref[:pos] # on enleve le '*' de la seq et tout ce qui est apres le '*'
            if cle in dict_count:
                dict_count[cle] += 1
            else:
                dict_count[cle] = 0
        except:
            cle = v_ref[:pos]
            dict_count[cle] = 0
    return dict_count, cle

In [30]:
# meme seq qu'avant mais on calcul en plus la distance
def ref_in_listoOfKeys_distance(listOfKeys, query, distance_max):
    dict_count = {}
    for v_ref in listOfKeys:
        distance = levenshtein_distance(v_ref, query )
        try:
            pos = v_ref.index('*')
            cle = v_ref[:pos]
            if cle in dict_count:
                dict_count[cle] += 1
            else:
                dict_count[cle] = 0
        except:
            cle = v_ref[:pos]
            dict_count[cle] = 0

        if distance > distance_max:
            distance_max = 0
            max_dic = cle
    return dict_count, max_dic, cle

In [36]:
# fonction pour enlever le '*' de la seq ref et tout ce qui est apres le '*'
def remove_etoile(tmp_d):
    try:
        pos = tmp_d.index('*')
        tmp_d = tmp_d[:pos]
    except:
        tmp_d = tmp_d
    return tmp_d

In [49]:
import time
def main_code(dic_db, dict_vquery_keys, dict_vquery_values, nb_indice = 60, bloc_max = 45, i =0, while_i = 100):

    #debut_algo = time.time()
    # gene ref
    dic_db_keys = list(dic_db.keys()) # on prend le nom du gene ref
    dic_db_values = list(dic_db.values()) # one prend la sequence du gene ref
    
    
    dic = {}
    tot_seq = 0 # le nombre de seqeunces etudiees
    tot_vrai = 0 # le nombre de vrais positifs
    temps_max = 0
    temps_levenshtein = 0
    
    # query
    #print(len(dict_vquery_keys))
    #print(len(dict_vquery_values))
    
    while i < while_i:
        
        tmp_d = dict_vquery_keys[i]
        query = dict_vquery_values[i]
        q_kmer = kmers_split(query, 5, 280, nb_indice) # on divise en [kmer, indice] la seq query

        debut_comp = time.time()
        # on commence l'algorithme
        dic = compare_fichier(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, k=5, taille_V = 280)
        fin_comp = time.time()
        temps_comp = fin_comp - debut_comp
        
        max_ = max(dic.items(), key=lambda x: x[1]) # on prend le score max des comparaisons faites
        listOfKeys = list()
        
        # on teste s'il y a plusieurs gene ref qui ont le meme score et on les mets dans une liste
        for key, value in dic.items():
            if value == max_[1]:
                listOfKeys.append(key)
        
        distance = 0
        distance_max = 0
        max_dic = ''
        
        # s'il y a bien plusieurs gene ref dans la liste alors on utilse la fct levenshtein_distance pour
        # essayer de trouver la bonne sequence
        dict_count = {} # pour compter le nombre de fois où on a le même gene d allèles differents
        dict_count, cle = ref_in_listoOfKeys(listOfKeys, query)
                
        if len(listOfKeys) == 1:
            temps_levenshtein = 0
            max_dic = cle
        
        if len(listOfKeys) > 1 :
            debut_levenshtein = time.time()
            dict_count, max_dic, cle = ref_in_listoOfKeys_distance(listOfKeys, query, distance_max)
                    
            fin_levenshtein = time.time()
            temps_levenshtein = fin_levenshtein - debut_levenshtein
        # on enleve le '*' de la seq ref et tout ce qui est apres le '*
        tmp_d = remove_etoile(tmp_d)
        print("Sequences avec même score :", listOfKeys)
        print("Nombre d'iteration        :", i)
        print("Sequence reference        :", max_dic)
        print("Sequence query            :", tmp_d)
        print("Temps levenshtein         :", temps_levenshtein)
        print("Temps complet:            :", temps_comp)
        
        temps_max += temps_comp
            
        
        # on test si on a des vrais positifs
        if max_dic == tmp_d: # si c'est des  vrais positifs
            print("-> VRAI Même sequence trouvée")
            tot_vrai += 1
        else:
            print("-> FAUX Différente sequence trouvée")
        print("-----------------------------------------------------")
        
        dic = {}
        tot_seq += 1
        i += 1
    vrai_pos = (tot_vrai/tot_seq)*100
    print("                   Résultat final                    ")
    print("-----------------------------------------------------")
    print("Nom du fichier ref                         :", db_path)
    print("Nom du fichier query                       :", name_fichier_query[46:len(name_fichier_query)])
    print("Nombre de vrais positifs                   :", tot_vrai)
    print("Nombre de séquences testées                :", tot_seq)
    print("Pourcentage de vrais positifs pour les V   :", vrai_pos, "%")
    print("Temps final de l'algorithme                :", strftime('%H %M %S', gmtime(temps_max)))
    print("-----------------------------------------------------")
    print("-----------------------------------------------------")
    
    return 0

In [50]:
# gene ref
db_path = "human_germline//IGHV.fasta"
dic_db = dict(read_db(db_path))

# gene query
name_fichier_query = 'VDJ-Kernel-main//data//datasets//simulations//simple_plus_indels_40Mut_out.fasta'
dict_vquery_keys, dict_vquery_values = extract_vquery(name_fichier_query)

start = time.perf_counter()
main_code(dic_db, dict_vquery_keys, dict_vquery_values)
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

Sequences avec même score : ['IGHV3-30*04', 'IGHV3-30-3*01', 'IGHV3-30-3*03']
Nombre d'iteration        : 0
Sequence reference        : IGHV3-30-3
Sequence query            : IGHV3-30-3
Temps levenshtein         : 8.821487426757812e-06
Temps complet:            : 0.5517950057983398
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV1/OR21-1*01']
Nombre d'iteration        : 1
Sequence reference        : IGHV1/OR21-1
Sequence query            : IGHV1/OR21-1
Temps levenshtein         : 0
Temps complet:            : 0.6930873394012451
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV3-30*10']
Nombre d'iteration        : 2
Sequence reference        : IGHV3-30
Sequence query            : IGHV3-30-3
Temps levenshtein         : 0
Temps complet:            : 0.7443599700927734
-> FAUX Différente sequence trouvée
-------------------------------------------------

Sequences avec même score : ['IGHV3-49*03', 'IGHV3-49*04', 'IGHV3-49*05']
Nombre d'iteration        : 26
Sequence reference        : IGHV3-49
Sequence query            : IGHV3-49
Temps levenshtein         : 9.059906005859375e-06
Temps complet:            : 0.4112372398376465
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV4-61*02']
Nombre d'iteration        : 27
Sequence reference        : IGHV4-61
Sequence query            : IGHV4-61
Temps levenshtein         : 0
Temps complet:            : 0.4541590213775635
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV3/OR16-6*02']
Nombre d'iteration        : 28
Sequence reference        : IGHV3/OR16-6
Sequence query            : IGHV3/OR16-6
Temps levenshtein         : 0
Temps complet:            : 0.49681997299194336
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequenc

Sequences avec même score : ['IGHV3/OR16-8*02']
Nombre d'iteration        : 52
Sequence reference        : IGHV3/OR16-8
Sequence query            : IGHV3/OR16-8
Temps levenshtein         : 0
Temps complet:            : 0.4561269283294678
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV7-81*01']
Nombre d'iteration        : 53
Sequence reference        : IGHV7-81
Sequence query            : IGHV7-81
Temps levenshtein         : 0
Temps complet:            : 0.522007942199707
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV1-69*04', 'IGHV1-69*09']
Nombre d'iteration        : 54
Sequence reference        : IGHV1-69
Sequence query            : IGHV1-69
Temps levenshtein         : 6.9141387939453125e-06
Temps complet:            : 0.36461591720581055
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même sc

Sequences avec même score : ['IGHV3-53*01']
Nombre d'iteration        : 78
Sequence reference        : IGHV3-53
Sequence query            : IGHV3-53
Temps levenshtein         : 0
Temps complet:            : 0.6128749847412109
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV1-8*01']
Nombre d'iteration        : 79
Sequence reference        : IGHV1-8
Sequence query            : IGHV1-8
Temps levenshtein         : 0
Temps complet:            : 0.4962601661682129
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV3-66*02']
Nombre d'iteration        : 80
Sequence reference        : IGHV3-66
Sequence query            : IGHV3-66
Temps levenshtein         : 0
Temps complet:            : 0.8752779960632324
-> VRAI Même sequence trouvée
-----------------------------------------------------
Sequences avec même score : ['IGHV4-4*05']
Nombre d'iteration        : 81

#### Tests pour determiner le meilleur indice et le meilleur block_max

In [16]:
def test(while_i):
    # on veut essayer de trouver le nb_indices et bloc_max les plus optimaux
    nb_indices = 55
    bloc_max = 35
    for ni in range(5):
        for bm in range(5):
            vrai_pos, temps = main_code(nb_indices, bloc_max, 0, while_i)
            print("nombres indices:", nb_indices, "bloc_max:", bloc_max, "vrai_pos:", vrai_pos, "temps:", temps)
            bloc_max += 5
            
        nb_indices += 5
        bloc_max = 35
        
    return 0
#test(50)